In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [14]:
# arquivo auxilliar (id_municipio e id_municipio_6)
municipio = pd.read_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/auxiliary_files/municipio.csv')


# import do arquivo original
df = pd.read_excel('/content/gdrive/MyDrive/ComprasPublicas_Brasil/auxiliary_files/Evolucao_por_Indicador_2013_a_2022_IFGF_2023.xlsx')
df = pd.merge(df, municipio[['id_municipio_6', 'id_municipio']], how='left', left_on='Código', right_on='id_municipio_6')

# transformação de wide pra long
vars = df.drop(['Código', 'UF', 'Município', 'id_municipio_6', 'id_municipio'], axis=1).columns.values
df1 = pd.melt(df, id_vars=['UF', 'id_municipio'], value_vars=vars).rename(columns={'UF':'sigla_uf'})

# ajuste das variáveis
df1['ano'] = df1['variable'].str[-4:]
df1['variable'] = df1['variable'].str[:-4]

# queries com base nas categorias: índice, ranking estadual e raking nacional
df11 = df1.query('variable == "IFGF "').rename(columns={'value':'indice_firjan_gestao_fiscal'}).drop('variable', axis=1)
df12 = df1.query('variable == "Ranking Estadual "').rename(columns={'value':'ranking_estadual'}).drop('variable', axis=1)
df13 = df1.query('variable == "Ranking IFGF "').rename(columns={'value':'ranking_nacional'}).drop('variable', axis=1)

# junção das queries gerando df no formato long
df_1 = pd.merge(df11, df12, how='left', on=['ano', 'sigla_uf', 'id_municipio'])
df_2 = pd.merge(df_1, df13, how='left', on=['ano', 'sigla_uf', 'id_municipio'])

# ordenação das variáveis
df_2 = df_2[['ano', 'sigla_uf', 'id_municipio', 'indice_firjan_gestao_fiscal', 'ranking_estadual', 'ranking_nacional']]

df_2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/auxiliary_files/ifgf.csv', index=False, na_rep='')